In [2]:
import tensorflow as tf
import numpy as np

2023-06-05 21:47:14.326188: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Initialize

We are storing the weight and bias matrices in python lists.  Each weight or bias matrix is a tf.Variable.  
We initialize each tf.Variable object by passing in two arguments: (1) its initial value and (2) its name.  
  
Here we want to initialize our weight and bias variables using (Xavier) Glorot normalization.  In general there are a number of choices under [tf.keras.initializers](https://www.tensorflow.org/api_docs/python/tf/keras/initializers):



In [47]:
def initialize_weights_and_biases(layer_dimensions):
    W, b = [], []
    initializer = tf.keras.initializers.GlorotNormal(seed=1)
    for i, (layer1, layer2) in enumerate(zip(layer_dimensions, layer_dimensions[1:]), 1):
        W.append(tf.Variable(initializer(shape=(layer1, layer2)), name = 'W' + str(i), dtype='float32'))
        b.append(tf.Variable(initializer(shape=(1, layer2)), name = 'b' + str(i), dtype='float32'))
    return {"W_i": W, "b_i": b}

We will set the activation function for each layer to relu, except for the last layer which will be a sigmoid activation function.  
  
These activation functions are present in [tf.keras.activations](https://www.tensorflow.org/api_docs/python/tf/keras/activations)

In [48]:
def relu_except_last_layer(layer_dimensions):
    return [tf.keras.activations.relu] * (len(layer_dimensions) - 2) + [tf.keras.activations.sigmoid]

Now we multiply weights by inputs using **tf.linalg.matmul** and add biases using **tf.math.add**

In [49]:
def forward_pass(W_i, b_i, g_i, X):
    for i, (W, b, g) in enumerate(zip(W_i, b_i, g_i)):
        if i == 0:
            Z = tf.math.add(tf.linalg.matmul(X, W), b)
        else:
            Z = tf.math.add(tf.linalg.matmul(A, W), b)
        A = g(Z)
    return A

## Example

### Data

In this example, positive class events are chosen from unit variance gaussian distribution in 10 dimensions that is displaced from the origin and negative class events are chosen from the unit variance gaussian distribution around the origin.

In [57]:
num_positive, num_negative, num_dims = 100, 900, 10
offset = np.random.randn(1, num_dims) / np.sqrt(num_dims)
positive = np.random.randn(num_positive, num_dims) + offset
negative = np.random.randn(num_negative, num_dims)
data = np.vstack((positive, negative))
data = np.hstack((data, np.vstack((np.ones((num_positive, 1)), np.zeros((num_negative, 1))))))
np.random.shuffle(data)
data, labels = data[:,:num_dims].astype('float32'), data[:, num_dims].astype('float32')

### Model

In [58]:
data.dtype

dtype('float32')

In [59]:
layers = [10, 10, 1]

parameters = initialize_weights_and_biases(layers)
W_i, b_i = [parameters[_] for _ in ["W_i", "b_i"]]
g_i = relu_except_last_layer(layers)
X = tf.constant(data)
forward_pass(W_i, b_i, g_i, X)

<tf.Tensor: shape=(1000, 1), dtype=float32, numpy=
array([[0.52742916],
       [0.5577931 ],
       [0.4702783 ],
       [0.28495017],
       [0.67950445],
       [0.3931118 ],
       [0.5389935 ],
       [0.5399908 ],
       [0.6048651 ],
       [0.6218038 ],
       [0.6394322 ],
       [0.6022722 ],
       [0.54646224],
       [0.6835919 ],
       [0.62702245],
       [0.49809483],
       [0.56212485],
       [0.5664179 ],
       [0.49577776],
       [0.71886516],
       [0.7210398 ],
       [0.715103  ],
       [0.29803976],
       [0.2880047 ],
       [0.46609464],
       [0.48629084],
       [0.48434702],
       [0.46700087],
       [0.36014244],
       [0.53593343],
       [0.31832182],
       [0.4593346 ],
       [0.59802896],
       [0.55750483],
       [0.12564133],
       [0.57562643],
       [0.57600594],
       [0.632271  ],
       [0.30183733],
       [0.562202  ],
       [0.53593343],
       [0.20109266],
       [0.5461822 ],
       [0.74036366],
       [0.5368922 ],
    

# Note`

We will want to compare with using tf.keras.layers.Dense(1)